In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import *
import pandas as pd

#Importation des données depuis un fichier 
data= pd.read_csv("C:/Users/Admin/Downloads/Données Devoir 3 LBM - Feuille 1.csv", engine='python')
#Copier le nom du chemin du dossier dans lequel le fichier de données "Données Devoir 3 LBM" se trouve à la place de 'C:/Users/Admin/Downloads/'

print(data) #Affichage des données récupérées du fichier texte
Dx = data[['dx (µm)']]
Permea_k = data[['k (µm2)']]
print()

#Conversion de la colonne de valeur dx en liste
DX=Dx.values.tolist()
dx_list = [] #initialisation de la liste
for i in range (0,len(DX)):
    dx_list += DX[i] #ajout de chaque élément dans la liste
    i += 1

#Conversion de la colonne de valeurs de perméabilités en liste
K=Permea_k.values.tolist()
K_list = [] #initialisation de la liste pour les valeurs de perméabilité
for i in range (0,len(K)):
    K_list += K[i]  #ajout de chaque élément dans la liste
    i += 1

r=2 #raffinement de maillage
pobs_list = [] #initialisation de la liste

#Calcul de l'ordre de convergence observé à partir de 3 maillages
for j in range(0,len(K_list)-2):
    pobs_list.append(log((K_list[j]-K_list[j+1])/(K_list[j+1]-K_list[j+2]))/(log(r)))
    j += 1

"Fonction permettant de déterminer l'incertitude pour la valeur SRQ du maillage le plus fin avec la formule du GCI" 
def détermination_GCI(pf,p_obs,k_grossier,k_fin,r):
    p=np.abs((p_obs-pf)/pf)
    if p <= 0.1:
        Fs= 1.25
        p = pf
    else:
        Fs= 3.0
        p = min(max(0.5,p_obs),pf)
    GCI= Fs*np.abs(k_grossier-k_fin)/(r**p-1)
    return GCI

pf=2 #Ordre de convergence formel en espace égal à 2 selon l'énoncé
r=2 #raffinement de maillage
GCI_list= ['NaN','NaN'] #initialisation de la liste

#Calcul de l'incertitude GCI (Grid Convergence Index) pour les valeurs de perméabilité k
for j in range(0,len(K_list)-2):
    GCI_list.append(détermination_GCI(pf,pobs_list[j],K_list[j+1],K_list[j+2],r)) #Concaténation des valeurs dans la liste GCI_list

unum_list=['NaN','NaN'] #initialisation de la liste
for i in range(2,len(GCI_list)):
    unum_list.append((GCI_list[i])/2) #Concaténation des valeurs des incertitudes numériques dans la liste unum_list   
    
#Tableau récapitulatif des résultats pour les incertitudes:
P_obs= pd.DataFrame(pobs_list, columns=['Ordre de convergence observé'])
GCI=pd.DataFrame(GCI_list, columns=['GCI'])
U_num=pd.DataFrame(unum_list, columns=['Unum'])
print(GCI)
print()
print("Tableau récapitulatif:")
pd.concat([data,P_obs,GCI,U_num],axis=1,ignore_index=False) #affichage du tableau récapitulatif (concaténation de toutes les colonnes)

    Seed  Raffinement maillage r    h   NX  dx (µm)  k (µm2)
0  100.0                       2  200   25      8.0  48.6990
1    NaN                       2  200   50      4.0  39.8744
2    NaN                       2  200  100      2.0  32.9173
3    NaN                       2  200  200      1.0  32.3299
4    NaN                       2  200  400      0.5  32.1127

         GCI
0        NaN
1        NaN
2  50.387776
3     0.5874
4     0.3823

Tableau récapitulatif:


,Seed,Raffinement maillage r,h,NX,dx (µm),k (µm2),Ordre de convergence observé,GCI,Unum
0,100.0,2,200,25,8.0,48.6990,0.343045,NaN,NaN
1,NaN,2,200,50,4.0,39.8744,3.566071,NaN,NaN
2,NaN,2,200,100,2.0,32.9173,1.435319,50.387776,25.193888
3,NaN,2,200,200,1.0,32.3299,NaN,0.5874,0.2937
4,NaN,2,200,400,0.5,32.1127,NaN,0.3823,0.19115
